# Introduction
In this notebook we do the derivation of the kinematics of the omnidrone for aerial tactile servoing. The final aim is to achieve a method for obtaining references for the controlled state velocities of the system (since the aerial platform is a quadrotor there is underactuation), which can be done by Jacobian (pseudo-)inverse methods. We regard the aerial manipulator as one system, thus the Jacobian is best derived from the centralised kinematics.

## Robot model
The robot is modelled as a kinematic chain where the first link is the drone platform, which is 'connected' to the environment by a 6DOF freedom joint (i.e. the drone can move independently from the environment). Then, the manipulator consists of three links (with lengths L1, L2, L3) with three revolute joints.

We use Denavit-Hartenberg for deriving the kinematics. Special attention needs to be paid to the intended workspace, because the Euler angle representation of the end-effector orientation implies singularities when the pitch (i.e. rotation about y) is $\pi/2 \vee -\pi/2$. It is wise to define reference frames (in particular, the manipulator base frame $\mathcal{F}_{0}$) such that these singularities are not close to the intended operational space in the workspace. We derive the kinematics in the homogeneous transformation matrix form, then we get the 6-dimensional kinematic function from the homogeneous transformation matrix. Finally, we can derive the Jacobian matrix by taking the partial derivative of each kinematic function element w.r.t. each state.

## State definition
The state of the aerial manipulator $\boldsymbol{\xi}$ is defined as
$$
\boldsymbol{\xi} = \begin{bmatrix} x_{b} & y_{b} & z_{b} & \psi_{b} & \theta_{b} & \varphi_{b} & q_{1} & q_{2} & q_{3}\end{bmatrix}^{\top}
$$
with $x_{b}$, $y_{b}$, and $z_{b}$ the position of the drone body CoM in the inertial frame, angles $\psi_{b}$, $\theta_{b}$, and $\varphi_{b}$ the Euler angles (yaw, pitch, roll) of the drone rotation with intrinsic ZYX convention, and $q_{1}$, $q_{2}$, and $q_{3}$ the angles of the revolute joints of the manipulator.

We start by setting up the relevant symbols and elementary rotation matrices with Sympy.

### Rotation convention
The yaw, pitch, roll (YPR) convention common in aerospace follows an intrinsic (that is, about the moving coordinate system) rotation about the body Z, Y, and X axes. 

For the calculation of the associated rotation matrices see the code below. This also shows that the YPR convention is equal to an extrinsic (that is, about the inertial coordinate system) rotation in the order XYZ.

The convention used here thus corresponds to the Euler ZYX mentioned in this source: http://web.mit.edu/2.05/www/Handout/HO2.PDF \
And this corresponds to the rotation matrix derived here: https://link.springer.com/article/10.1007/s11071-022-08212-w

In [2]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)

L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3 = sp.symbols('x_b y_b z_b yaw pitch roll q_1 q_2 q_3', real=True)
state_vector = sp.Matrix([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

# Secondary states
x_be, y_be, z_be = sp.symbols('x^{b}_{be} y^{b}_{be} z^{b}_{be}')

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

# Elementary rotation matrices
def Rx(x):
    return sp.Matrix([[1, 0, 0], [0, sp.cos(x), -sp.sin(x)], [0, sp.sin(x), sp.cos(x)]])

def Ry(y):
    return sp.Matrix([[sp.cos(y), 0, sp.sin(y)], [0, 1, 0], [-sp.sin(y), 0, sp.cos(y)]])

def Rz(z):
    return sp.Matrix([[sp.cos(z), -sp.sin(z), 0], [sp.sin(z), sp.cos(z), 0], [0, 0, 1]])

### Frame definitions
The nominal configuration (i.e. all joint angles 0) is defined as the arm pointing upwards and being fully extended. The frames for this derivation are defined as follows:
- **Inertial frame** $\mathcal{F}_{I}$ - Frame attached to the world, unmoving, North-East-Down for positive XYZ axes.
- **Body frame** $\mathcal{F}_{b}$ - Frame attached to the drone body in its center of mass (CoM), moves and rotates with body, Forward-Right-Down for positive XYZ axes
- **Manipulator base frame** $\mathcal{F}_{0}$ - Origin coincides with body frame, but rotated to Up-Right-Forward for positive XYZ to align Z axis with rotation of $q_{1}$ and set Y axis such that Euler angle singularity in $\mathcal{F}_{0}$ is placed away from operational workspace. Does not rotate with $q_{1}$
- **Frame 1** $\mathcal{F}_{1}$ - Z-axis coincides with axis of rotation of $q_{2}$. X points towards $q_{3}$ when $q_{2}=0$. Does not rotate with $q_{2}$
- **Frame 2** $\mathcal{F}_{2}$ - Z-axis coincides with axis of rotation of $q_{3}$. X points towards end-effector. Does not rotate with $q_{3}$
- **Frame 3** $\mathcal{F}_{3}$ - Oriented equal to $\mathcal{F}_{2}$ when $q_{3}=0$. Origin at end-effector. 
- **Frame e** $\mathcal{F}_{e}$ - Oriented with positive Z pointing outwards, X in direction of negative $Z_{3}$ and Y completing right handed

<p align="center">
    <img title="Kinematic diagram" src="images/kinematic_diagram.png">
</p>
### Angular velocity
Taking the time-derivative of expressions of Euler angles does not yield angular velocities. A transformation matrix exists to convert between them. For the YPR convention described above, this looks like:
$$
T(\phi_{b})=\begin{bmatrix} 0 & -\sin(\psi) & \cos(\psi) \cos(\theta) \\
                            0 & \cos(\psi) & \sin(\psi) \cos(\theta) \\
                            1 & 0 & 0 \end{bmatrix}
$$

Source: https://ieeexplore.ieee.org/document/6608869

In [3]:
# Setting up transformation for free body
Rz_yaw = Rz(yaw)
Ry_pitch = Ry(pitch)
Rx_roll = Rx(roll)

# Intrinsic ZYX: Postmultiplication
R_Ib = Rz_yaw * Ry_pitch * Rx_roll

# Transformation between euler angle time derivatives and angular velocity
T_euler = sp.Matrix([[0, -sp.sin(yaw), sp.cos(yaw)*sp.cos(pitch)],
                     [0, sp.cos(yaw), sp.sin(yaw)*sp.cos(pitch)],
                     [1, 0, -sp.sin(pitch)]])

T_A = sp.Matrix([[sp.eye(3), sp.zeros(3, 3)],
                 [sp.zeros(3, 3), T_euler]])

# Homogeneous transformation matrix for inertial to body frame (i.e. 6DOF freedom joint)
H_Ib = sp.Matrix([[sp.cos(yaw)*sp.cos(pitch), sp.cos(yaw)*sp.sin(pitch)*sp.sin(roll)-sp.sin(yaw)*sp.cos(roll), sp.cos(yaw)*sp.sin(pitch)*sp.cos(roll)+sp.sin(yaw)*sp.sin(roll), x_b],
                  [sp.sin(yaw)*sp.cos(pitch), sp.sin(yaw)*sp.sin(pitch)*sp.sin(roll)+sp.cos(yaw)*sp.cos(roll), sp.sin(yaw)*sp.sin(pitch)*sp.cos(roll)-sp.cos(yaw)*sp.sin(roll), y_b],
                  [-sp.sin(pitch), sp.cos(pitch)*sp.sin(roll), sp.cos(pitch)*sp.cos(roll), z_b],
                  [0, 0, 0, 1]])

In [4]:
# Denavit-Hartenberg approach
# Parameters
DH_theta = [q1, q2, q3]
DH_alpha = [-sp.pi/2, sp.pi/2, 0]
DH_r = [L1, L2, L3]
DH_d = [0, 0, 0]

# Manipulator base frame is rotated w.r.t. body frame but origins coincide
H_b0 = sp.Matrix([[ 0, 0, 1, 0],
                  [ 0, 1, 0, 0],
                  [-1, 0, 0, 0],
                  [ 0, 0, 0, 1]])

# Frame 0: X pointing right, Z pointing forward, Y pointing down (i.e. rotation around Zb by pi/2, then rotation around Xb' by pi/2)
# I.e. intrinisic rotation so multiply Rz * Rx'
# Frame 1: Attached at Q2, Z pointing down, X pointing right (rotation around X0 of -pi/2, then translation along x0' by L1)

H_01 = sp.Matrix([[sp.cos(DH_theta[0]), -sp.sin(DH_theta[0])*sp.cos(DH_alpha[0]), sp.sin(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.cos(DH_theta[0])],
                  [sp.sin(DH_theta[0]), sp.cos(DH_theta[0])*sp.cos(DH_alpha[0]), -sp.cos(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.sin(DH_theta[0])],
                  [0, sp.sin(DH_alpha[0]), sp.cos(DH_alpha[0]), DH_d[0]],
                  [0, 0, 0, 1]])

# Frame 2: Attached at Q3, Z pointing forward, X pointing right (rotation around X1 of pi/2, then translation along x1' by L2)
H_12 = sp.Matrix([[sp.cos(DH_theta[1]), -sp.sin(DH_theta[1])*sp.cos(DH_alpha[1]), sp.sin(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.cos(DH_theta[1])],
                  [sp.sin(DH_theta[1]), sp.cos(DH_theta[1])*sp.cos(DH_alpha[1]), -sp.cos(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.sin(DH_theta[1])],
                  [0, sp.sin(DH_alpha[1]), sp.cos(DH_alpha[1]), DH_d[1]],
                  [0, 0, 0, 1]])

# Frame 3: Attached at EE, Z pointing forward, X pointing right (translation along x2 by L3)
H_23 = sp.Matrix([[sp.cos(DH_theta[2]), -sp.sin(DH_theta[2])*sp.cos(DH_alpha[2]), sp.sin(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.cos(DH_theta[2])],
                  [sp.sin(DH_theta[2]), sp.cos(DH_theta[2])*sp.cos(DH_alpha[2]), -sp.cos(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.sin(DH_theta[2])],
                  [0, sp.sin(DH_alpha[2]), sp.cos(DH_alpha[2]), DH_d[2]],
                  [0, 0, 0, 1]])

# Transformation between 3 and e - To align e with frame e from direct derivation
H_3e = sp.Matrix([[ 0,  0, 1, 0],
                  [ 0,  1, 0, 0],
                  [-1,  0, 0, 0],
                  [ 0,  0, 0, 1]])

# Equivalent transformation
H_be = (H_b0 * H_01 * H_12 * H_23 * H_3e).subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

In [5]:
# Testing if the DH formulation yielded the same result as the direct derivation
# Unit test 1
q1_val = 0.0
q2_val = 0
q3_val = 0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if (H_be_subs[2,3] + (L1_length+L2_length+L3_length)< 0.0001):
    print("Unit test 1 passed")
else:
    print("Unit test 1 failed")

# Unit test 2
q1_val = sp.pi/2
q2_val = 0
q3_val = 0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if (H_be_subs[1,3] - (L1_length+L2_length+L3_length)< 0.0001):
    print("Unit test 2 passed")
else:
    print("Unit test 2 failed")

# Unit test 3
q1_val = sp.pi/2-sp.pi/6
q2_val = 0.0
q3_val = sp.pi/6
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if ((H_be_subs[1,3] - (sp.cos(sp.pi/6).evalf()*(L1_length+L2_length)+L3_length)< 0.0001) and (H_be_subs[2,3] + (sp.sin(sp.pi/6).evalf()*(L1_length+L2_length))< 0.0001)):
    print("Unit test 3 passed")
else:
    print("Unit test 3 failed")

# Unit test 4
q1_val = sp.pi/2
q2_val = sp.pi/6
q3_val = 0.0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if ((H_be_subs[0,3] + (sp.sin(sp.pi/6).evalf()*(L3_length+L2_length))< 0.0001) and (H_be_subs[1,3] - (sp.cos(sp.pi/6).evalf()*(L3_length+L2_length)+L1_length)< 0.0001)):
    print("Unit test 4 passed")
else:
    print("Unit test 4 failed")

Unit test 1 passed
Unit test 2 passed
Unit test 3 passed
Unit test 4 passed


### Manipulator kinematics
First we derive the kinematics function of the manipulator in $\mathcal{F}_{0}$. This is needed for the manipulator position controller that is used for homing and positioning the manipulator outside of the contact operation.
Since we have 3 degrees of freedom in the manipulator, we can only control three out of the six total pose components. Intuitively, we can immediately see that control of the Z coordinate is delegated to $q_{2}$, and it coupled with the roll (rotation about $x_{0}$). We choose to control this DoF by defining the roll angle.
Next, we can see that the yaw (rotation around $z_{0}$) is dependent on a combination of $q_{1}$ and $q_{3}$. This is coupled with the $x_{0}$ and $y_{0}$ positions.
Finally, the rotation around $y_{0}$ is dependent on $q_{2}$ as well, but this is only controllable in certain configurations.

Then, how can we devise an inverse kinematics algorithm for this? 

In [12]:
# Forward kinematics function of the manipulator (in the body frame)
FK_manipulator = sp.Matrix([H_be[0,3],
                H_be[1,3],
                H_be[2,3],
                sp.atan2(H_be[1,0], H_be[0,0]), # yaw
                -sp.asin(H_be[2,0]), # pitch
                sp.atan2(H_be[2,1], H_be[2,2])]) # roll

FK_manipulator = sp.simplify(sp.trigsimp(FK_manipulator))
sp.pprint(FK_manipulator)
for i in range(len(FK_manipulator)):
    print("FK_manipulator[", i, "] = f(", FK_manipulator[i].free_symbols, ")")

yaw_man_e, pitch_man_e, roll_man_e, x_man_e, y_man_e, z_man_e = sp.symbols("yaw_0 pitch_0 roll_0 x_0 y_0 z_0")
sol = sp.solve([FK_manipulator[3]-yaw_man_e, FK_manipulator[4]-pitch_man_e], q1, q2)
sp.pprint(sol)

q1_val = sp.pi/2-sp.pi/6
q2_val = 0.0
q3_val = sp.pi/6
FK_manipulator_subs = FK_manipulator.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
print("Forward kinematics of the manipulator (in the body frame)")
sp.pprint(FK_manipulator_subs)

⎡                              -(0.273⋅cos(q₃) + 0.311)⋅sin(q₂)                ↪
⎢                                                                              ↪
⎢0.273⋅sin(q₁)⋅cos(q₂)⋅cos(q₃) + 0.311⋅sin(q₁)⋅cos(q₂) + 0.11⋅sin(q₁) + 0.273⋅ ↪
⎢                                                                              ↪
⎢0.273⋅sin(q₁)⋅sin(q₃) - 0.273⋅cos(q₁)⋅cos(q₂)⋅cos(q₃) - 0.311⋅cos(q₁)⋅cos(q₂) ↪
⎢                                                                              ↪
⎢                             atan2(-sin(q₁)⋅sin(q₂), -cos(q₂))                ↪
⎢                                                                              ↪
⎢                                   -asin(sin(q₂)⋅cos(q₁))                     ↪
⎢                                                                              ↪
⎣atan2(sin(q₁)⋅cos(q₃) + sin(q₃)⋅cos(q₁)⋅cos(q₂), sin(q₁)⋅sin(q₃) - cos(q₁)⋅co ↪

↪                ⎤
↪                ⎥
↪ sin(q₃)⋅cos(q₁)⎥
↪                ⎥
↪  - 0.11⋅cos(q₁)⎥
↪            

KeyboardInterrupt: 

In [7]:
# Forward kinematics function of the combined kinematics (in the inertial frame)
H_Ie = H_Ib * H_be
FK_centralised = sp.Matrix([H_Ie[0,3],
                H_Ie[1,3],
                H_Ie[2,3],
                sp.atan2(H_Ie[1,0], H_Ie[0,0]), # yaw
                -sp.asin(H_Ie[2,0]), # pitch
                sp.atan2(H_Ie[2,1], H_Ie[2,2])]) # roll
FK_centralised = sp.trigsimp(sp.expand(FK_centralised))

sp.pprint("State dependencies of centralised kinematics")
print(f"xe = f({FK_centralised[0,0].free_symbols})")
print(f"ye = f({FK_centralised[1,0].free_symbols})")
print(f"ze = f({FK_centralised[2,0].free_symbols})")
print(f"yawe = f({FK_centralised[3,0].free_symbols})")
print(f"pitche = f({FK_centralised[4,0].free_symbols})")
print(f"rolle = f({FK_centralised[5,0].free_symbols})")

sp.pprint("\nDenominators for yaw and roll")
sp.pprint(H_be[0,0])
sp.pprint(H_be[2,2])

# Jacobian calculation for velocity
J_A_centralised = FK_centralised.jacobian([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

print(f"\nCentralised Jacobian shape: {J_A_centralised.shape}")

# Instantaneous states
xb_inst = 0.0
yb_inst = 0.0
zb_inst = 0.0
yaw_inst = 0.0
pitch_inst = 0.0
roll_inst = 0.0
q1_inst = sp.pi/2-sp.pi/6
q2_inst = 0
q3_inst = sp.pi/6

# Substitute the instantaneous states
J_A_centralised_inst = J_A_centralised.subs([(x_b, xb_inst), (y_b, yb_inst), (z_b, zb_inst), (yaw, yaw_inst), (pitch, pitch_inst), (roll, roll_inst), (q1, q1_inst), (q2, q2_inst), (q3, q3_inst)]).evalf()
if J_A_centralised_inst.rank() == 6:
    print(f"The Jacobian is full rank (rank={J_A_centralised_inst.rank()})")

State dependencies of centralised kinematics
xe = f({q_3, x_b, roll, q_2, q_1, pitch, yaw})
ye = f({y_b, q_3, roll, q_2, q_1, pitch, yaw})
ze = f({q_3, roll, q_2, q_1, pitch, z_b})
yawe = f({roll, q_2, q_1, pitch, yaw})
pitche = f({q_1, roll, q_2, pitch})
rolle = f({q_3, roll, q_2, q_1, pitch})
                             
Denominators for yaw and roll
-cos(q₂)
sin(q₁)⋅sin(q₃) - cos(q₁)⋅cos(q₂)⋅cos(q₃)

Centralised Jacobian shape: (6, 9)
The Jacobian is full rank (rank=6)


Could do kinematic singularity analysis here but maybe not necessary

## Inverse kinematics
Now that we have derived the Jacobian and it's not singular close to our desired workspace, we can define the inverse kinematics law for the system.

In [15]:
# Closed loop inverse kinematics
print(f"Jacobian = J({J_A_centralised.free_symbols})")

J_G_centralised = T_A*J_A_centralised

jacobian_object_name = "self.jacobian"
for i in range(J_G_centralised.shape[0]):
    for j in range(J_G_centralised.shape[1]):
        if J_G_centralised[i,j] != 0:
            print(f"{jacobian_object_name}[{i},{j}]={J_G_centralised[i,j]}")
               

J_G_controlled = J_G_centralised[:,0:4].row_join(J_G_centralised[:,6:9])
J_G_uncontrolled = J_G_centralised[:,4:6]

Jacobian = J({q_3, roll, q_2, q_1, pitch, yaw})
self.jacobian[0,0]=1.00000000000000
self.jacobian[0,3]=-0.273*sin(pitch)*sin(q_3)*sin(yaw)*sin(q_1 + roll) + 0.273*sin(pitch)*sin(yaw)*cos(q_2)*cos(q_3)*cos(q_1 + roll) + 0.311*sin(pitch)*sin(yaw)*cos(q_2)*cos(q_1 + roll) + 0.11*sin(pitch)*sin(yaw)*cos(q_1 + roll) + 0.273*sin(q_2)*sin(yaw)*cos(pitch)*cos(q_3) + 0.311*sin(q_2)*sin(yaw)*cos(pitch) - 0.273*sin(q_3)*cos(yaw)*cos(q_1 + roll) - 0.273*sin(q_1 + roll)*cos(q_2)*cos(q_3)*cos(yaw) - 0.311*sin(q_1 + roll)*cos(q_2)*cos(yaw) - 0.11*sin(q_1 + roll)*cos(yaw)
self.jacobian[0,4]=0.273*sin(pitch)*sin(q_2)*cos(q_3)*cos(yaw) + 0.311*sin(pitch)*sin(q_2)*cos(yaw) + 0.273*sin(q_3)*sin(q_1 + roll)*cos(pitch)*cos(yaw) - 0.273*cos(pitch)*cos(q_2)*cos(q_3)*cos(yaw)*cos(q_1 + roll) - 0.311*cos(pitch)*cos(q_2)*cos(yaw)*cos(q_1 + roll) - 0.11*cos(pitch)*cos(yaw)*cos(q_1 + roll)
self.jacobian[0,5]=0.273*sin(pitch)*sin(q_3)*cos(yaw)*cos(q_1 + roll) + 0.273*sin(pitch)*sin(q_1 + roll)*cos(q_2)*cos(q_3)*cos

The initial idea was to derive symbolically the differential equation that is being reported (Arleo, 2013):
$$
\dot{\zeta}_{r}=J^{\dagger}_{\zeta}(\sigma_{b}, \zeta_{r})T_{A}(\Phi_{e,r})(\dot{x}_{e,d}+Ke)-J^{\dagger}_{\zeta}(\sigma_{b}, \zeta_{r})J_{\sigma}(\sigma_{b}, \zeta_{r})\dot{\sigma}_{b} \\
J^{\dagger}_{\zeta} = J_{\zeta}^{\top}(J_{\zeta}J_{\zeta}^{\top})^{-1} \\
e = x_{e,d} - x_{e,r}
$$
However this is impractical due to the complexity of the inversion operation. There are several other approaches that could supplement this:
- Given that the system cannot track both generalized coordinates and their derivates at the same time, get the measured values of the current generalized coordinates and use these as the input to the equation. Then pass the derivatives back as references.
- Employ a position inverse kinematics scheme to find $\zeta_{r}$ from $x_{e,d}$, the desired end-effector pose (note that $x_{e,r}$ denotes the end-effector pose computed on the basis of $\zeta_{r}$)

Let's investigate these options.

### Algorithm for option 1
We follow the formulation by Siciliano 2009, page 133, with the partitioning into controllable and uncontrollable parts as in Arleo 2013.
1. We set up the geometric Jacobian symbolically $J_{G}(\xi)$. This is programmed as a callable function returning the evaluated Jacobian matrix. We split the Jacobian into the controllable and uncontrollable Jacobians by columns.
2. We measure or estimate all states. The servo positions can be measured from the encoders, the drone orientation from the Kalman filter. Note that the Jacobian is not a function of drone position, so we don't need to estimate this.
3. Evaluate $J_{A}$.
4. Calculate forward kinematics to find $x_{e}$.
5. Calculate error $e=x_{d}-x_{e}$, with $x_{d}$ the desired end-effector setpoint.
6. Multiply error by gain matrix $K$ and add to $\dot{x}_{d}$, the desired end-effector velocity: $\dot{x}_{d}+K(x_{d}-x_{e})$
7. Invert numerical Jacobian and multiply by the error term to obtain references for the states: $\dot{\zeta}=J_{a}^{-1}(\xi)$
8. Feed $\dot{\xi}$ as reference to the actuators.
9. Repeat.

**Notes**
- One of the advantages of this approach is that we do not need to rely on position estimates of the drone, which are notoriously difficult to get accurate in outdoor conditions. The other states can be estimated with exceeding precision.
- The instantaneous reference state is the current state position, and a targeted state velocity. Therefore, we can use __measured__ states to evaluate the Jacobian.